# Lesson 8 - Source Sheet

A source sheet is an infinite row of sources along one line

In [1]:
# import resources notebook
%run resources.ipynb

In [2]:
# create mesh grid
N = 100
x_start, x_end = -1.0, 1.0
y_start, y_end = -1.5, 1.5
x, y = np.linspace(x_start, x_end), np.linspace(y_start, y_end)
X, Y = np.meshgrid(x, y)

In [3]:
# create freestream
u_inf = 1.0
alpha = 0.0

fs = Freestream(u_inf, alpha)
fs.velocity(X, Y)
fs.stream_function(X, Y)

## Finite number of sources along a line

In [ ]:
N_sources = 11
strength = 5
strength_source = strength/N_sources
x_source = np.zeros(N_sources, dtype=float)
y_source = np.linspace(-1.0, 1.0, N_sources)

# create a source line
sources = np.empty(N_sources, dtype=object)